# DATA3800 - Introduction to data science with scripting
## Predictively modeling salmon louse counts in fish farming - *project code*

This Jupyter notebook contains all the code for the project in a runnable format. Note that the descriptions of the various steps is not as in-depth her, since I covered that in the report.

Running R in a Jupyter notebook can be a bit finicky, since it requires that we run both a `venv` (Python virtual environment) and a `renv` (R virtual environment). R also doesn't like a nested project structure, so there are a few extra steps involved with getting the IRkernel to recognise the renv. I have done my best to make sure everything is in order, but just in case the Jupyter notebook isn't cooperative, I have also included the raw R script file under the `backup_script/` directory.

### Preparations - Load packages, source data, and set seed:

I start by loading the necessary packages for running the script. Several additional packages will be called directly using the `package_name::function_name()` syntax. I've told the `library()` function to load packages quietly for the sake of clarity, since the outputs can take a lot of space when viewed in a Jupyter notebook.

In [1]:
# Load necessary packages:
library(dplyr, quietly = TRUE)
library(tidyr, quietly = TRUE)
library(stringr, quietly = TRUE)
library(lme4, quietly = TRUE)

ERROR: Error in library(dplyr, quietly = TRUE): there is no package called ‘dplyr’


By setting a seed (in this case a cheeky reference to the course code) I ensure that the numbers generated by the pseudorandom number generator stay consistent between different sessions (though they may differ between different pseudorandom number generator and the operative systems which use them).

In [ ]:
# Set seed to ensure reproducibility:
set.seed(3800)

In [ ]:
# Source data files:
lice_per_fish  <- readr::read_csv("../data/lakselus_per_fisk.csv")
lice_treatment <- readr::read_csv("../data/tiltak_mot_lakselus.csv")

### Data wrangling:

This is where the fun begins. First we must manipulate the data into a usable state.

In [ ]:
# Replace spaces with underscores in all column names and make lowercase:
list(lice_per_fish = lice_per_fish,
     lice_treatment = lice_treatment) %>% 
  lapply(function(df) {
    rename_all(df, ~tolower(.)) %>%                 # Change to lowercase.
    rename_all(.,  ~str_replace_all(., " ", "_"))   # Spaces to underscores.
  }) %>% 
  list2env(envir = .GlobalEnv)  # Save to global environment.

Cleanerfish were only recorded until the start of 2019, after which the registrations were discontinued. The first of the code snippets below returns a table view containing the number of times each species of cleanerfish appear within the dataset. The latter snippet returns the number of unique species, or in this case: NA :(

In [ ]:
# Exploring the presence of cleanerfish species before and after 31. December, 2018:

# Before change:
lice_treatment %>% 
  filter(år <= 2018) %>% 
  select(rensefisk) %>% 
  table()

# After change:
lice_treatment %>% 
  filter(år > 2018) %>% 
  select(rensefisk) %>% 
  unique()

Before I can join the two data frames together, I must prepare the data frame 'lice_treatment' for joining. When I `left_join()` `lice_treatment` to `lice_per_fish`, it inserts only rows which matches observations in `lice_per_fish`. In this case, since one farm site can recieve different treatments even within the same week, we get duplicate records when we look at just the shared columns. To deal with this, we combine all the records from the same site, and spread the data out into new columns.

In [ ]:
# Prepare data frame 'lice_treatment' for joining:
lice_treatment <- lice_treatment %>% 
  select(uke, år, lokalitetsnavn, tiltak) %>% 
  
  # Remove duplicate rows:
  distinct() %>% 
  
  # Give each of the treatment categories its own boolean column:
  mutate(value = TRUE) %>% 
  pivot_wider(
    names_from = tiltak,  # Column to be split.
    values_from = value,
    values_fill = FALSE)

Now we join the two data frames together, before we manipulate the resulting data frame further. The first thing we do after combining the data is removing rows for weeks where a salmon louse count was not carried out, since these observations are irrelevant to the project. Next, we fill the boolean columns created in the last cell with `FALSE` values, so that we don't get any missing data errors later. Finally, we add the columns for cyclically transformed time and the log transformed response variable. Sorting the rows is done just for my convenience in exploring the data.

In [ ]:
# Join the dataframe 'lice_treatment' to 'lice_per_fish':
working_df <- left_join(lice_per_fish,
                        lice_treatment) %>% 
  
  # Remove rows with no salmon lice counts:
  subset(har_telt_lakselus == "Ja") %>% 
  
  # Fill NAs in the newly added boolean columns with FALSE:
  mutate_if(is.logical, coalesce, FALSE) %>% 
  
  # Change week 53 to week 52 for all records:
  mutate(uke = if_else(uke == 53, 52, uke)) %>% 
  
  # Add variables for cyclic time:
  {
    # Assign local variable 'angle':
    angle = (2 * pi * .$uke) / 52 
    
    # Calculate flipped sine and cosine transformed values:
    mutate(., sin = -sin(angle),
              cos = -cos(angle))
  } %>%
  
  # Create the log transformed response variable:
  {
    const <- 0.01  # Add small constant to handle zeros.
    
    # NB! log() in R refers to the natural logarithm.
    mutate(., log_response = log(voksne_hunnlus + const))
  } %>% 
  
  # Arrange rows chronologically:
  arrange(år, uke, lokalitetsnavn)

To decide on which model to use, I had to explore the data in order to see if it conformed to the model assumptions.

### Analysis

This is the general syntax to fit a model with `lmer()`, with several fixed effects and `lokalitetsnavn` as a random effect. In this example, for the sake of illustration, I have included each of the considered predictor variables and interaction effects:

In [ ]:
# Fit the Linear Mixed Model:
model <- lmer(log_response ~ sjøtemperatur +
                             lat +
                             cos + sin +
                             sjøtemperatur * cos +
                             sjøtemperatur * sin +
                             sjøtemperatur * lat +
                             cos * lat +
                             sin * lat +
                             (1 | lokalitetsnavn),
              data = train)

# Apply model to the test set:
test <- test %>% 
  # Save the fitted response to a new column:
  mutate(fitted_log_response = predict(model, 
                                       newdata = test,
                                       allow.new.levels = TRUE),
         
         # Back-transform the fitted response to avg. count:
         fitted_voksne_hunnlus = exp(fitted_log_response) - 0.01)

# Calculate metrics for model performance evaluation: 
rmse_value <- Metrics::rmse(test$fitted_log_response,  # Root Mean Square Error.
                            test$log_response)
mae_value  <- Metrics::mae(test$fitted_log_response,   # Mean Absolute Error.
                           test$log_response)
r_squared  <- MuMIn::r.squaredGLMM(model)              # Approximated R squared.

To find the best model, we must create and evaluate each candidate model and select the model which prerforms the best. We start by defining candidate models:

In [ ]:
# Define variable fixed effects:
# NB! Sea temperature is always included.
predictors <- c(
  "lat",
  "cos",
  "sin",
  "sjøtemperatur * lat",
  "sjøtemperatur * cos",
  "sjøtemperatur * sin",
  "cos * lat",
  "sin * lat"
)

# Generate combinations of predictors:
predictor_combinations <- 
  lapply(1:length(predictors), 
         function(n) {
           combinations <- gtools::combinations(length(predictors), 
                                                n, 
                                                as.character(predictors))
           lapply(1:nrow(combinations), 
                  function(i) combinations[i, ])})

# Simplify the data structure:
predictor_combinations <- unlist(predictor_combinations, 
                                 recursive = FALSE)

Next, we run the list of potential candidate models through a function to strip away duplicates:

In [ ]:
# Function to remove strings that are substrings of another string:
remove_substrings <- function(lst) {
  lst[!vapply(seq_along(lst), 
              function(i) any(grepl(lst[i], 
                                    lst[-i])), 
              logical(1))]
  }

# Apply the function to the combinations of predictors:
predictor_combinations <- predictor_combinations %>% 
  
  # Remove substrings:
  lapply(remove_substrings) %>% 
  
  # Remove duplicate list entries:
  unique()

Next, we run through the candidate models and save the models to a list:

In [ ]:
# Create a list to store model results:
models <- list()

# Iterate through the combinations list and fit models for each combination:
# Grab a cup of coffee, this will take a little while!
for (combination in predictor_combinations) {
  
  # Specify the formula based on the list of combinations:
  formula <- as.formula(paste("log_response ~ sjøtemperatur +
                                              (1 | lokalitetsnavn) +",
                              paste(combination, collapse = " + ")))
  
  # Run the model:
  model <- lmer(formula = formula, data = train)
  
  # Save the model to the list we created:
  models[[paste(c("sjøtemperatur", 
                  combination), collapse = " + ")]] <- model
}

Now we run the models through the test set and record the results of the evaluation:

In [ ]:
# Create the data frame to store the evaluation metrics:
model_evaluation <- data.frame(model = names(models),
                               rmse  = NA,
                               mae   = NA,
                               r2m   = NA,
                               r2c   = NA)

for (i in seq_along(models)) {
  
  # Run the model through the test set:
  prediction <- predict(models[[i]],
                        newdata = test,
                        allow.new.levels = TRUE)
  
  # Calculate RMSE:
  model_evaluation[i, "rmse"] <- Metrics::rmse(prediction,
                                             test$log_response)
  # Calculate MAE:
  model_evaluation[i, "mae"] <- Metrics::mae(prediction,
                                           test$log_response)
  # Calculate R^2:
  model_evaluation[i, c("r2m", "r2c")] <- MuMIn::r.squaredGLMM(models[[i]]) %>% 
    as.vector()
}

The following block finds the best model according to each metric:

In [ ]:
# Find best model according to all evaluation metrics:
model_evaluation %>%
  filter(rmse == min(rmse))  # Find minimum RMSE value.

model_evaluation %>%
  filter(mae == min(mae))    # Find minimum MAE value.

model_evaluation %>%
  filter(r2m == max(r2m))    # Find maximum R^2m value.

model_evaluation %>%
  filter(r2c == max(r2c))    # Find maximum R^2c value.

### Plot code:

The following code produces the visualisation of the sine and cosine transformed cyclic time variables. For the sake of convenience, I have modified to function to output the plot to the default graphics device. Simply comment the relevant sections back in to have it write to file.

In [ ]:
# Create a visualisation of the cyclically transformed time variable:
working_df %>% 
  select(uke, sin, cos) %>% 
  arrange(uke) %>% 
  {
    # TO WRITE TO FILE, COMMENT IN THIS:
      
    # # Specify where to save the finished plot:
    # pdf("../figures/cyclic_time.pdf",
    #     width  = 8,
    #     height = 5)
    #
    # # Adjust padding:
    # par(mar = c(4, 4, 2, 2))
    
    attach(.)  # Lets me select columns by name without indexing.
    
    # Plot the sin curve:
    plot(sin~uke, 
         type = "l", 
         lwd = 2, 
         cex.lab = 1.2,
         col = "orange",
         ylab = "Yearly oscillation", 
         xlab = "Week number")
    
    # Plot the cos curve:
    lines(cos~uke, 
          lwd = 2, 
          col = "dimgray")
    
    # Add a legend:
    legend("topleft", 
           inset = c(0.05,0.08),
           legend = c("sin 1", "cos 1"), 
           col = c("orange", "dimgray"), 
           lwd = 2, 
           cex = 1.2)
    
    detach(.)  # Undo the previous attach() command.

    # AND THIS:
      
    # # Save the plot to PDF:
    # dev.off()
  }

Create the histogram of the residual distribution:

In [ ]:
# Histogram of the residuals:

# Find the best model (any metric will do, as they all agree):
model_evaluation %>%
      filter(r2c == max(r2c)) %>% select(model)

hist_model <- models[["sjøtemperatur_cos * lat_sin * lat_sjøtemperatur * cos_sjøtemperatur * lat_sjøtemperatur * sin"]]

hist(residuals(hist_model), 
     freq = FALSE,
     xlab = "Residuals",
     main = "Distribution of the residuals")

Explore the distribution of the data. The data are strongly right-skewed:

In [ ]:
# Generate a sequence of values based on the fitted distribution
x <- seq(0, max(working_df$voksne_hunnlus + 0.01), length.out = 1000)

# Plot histogram of the data
hist(working_df$voksne_hunnlus + 0.01, freq = FALSE, xlim = c(0, max(2)), breaks = 200,
     main = "Fitted Log-Normal distribution",
     xlab = "Distribution")

# Add the fitted log-normal density curve
curve(dlnorm(x, meanlog = fit$estimate["meanlog"], sdlog = fit$estimate["sdlog"]), 
      col = "blue", lwd = 2, add = TRUE)